In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

In [ ]:
t_cells <- readRDS(file = 'aging_t_cells.rds')

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)
DimPlot(t_cells, label = TRUE, repel = TRUE, label.size = 10)
DimPlot(t_cells, group.by = 'age', shuffle = TRUE)

In [ ]:
DefaultAssay(t_cells) <- 'RNA'
t_cells <- NormalizeData(t_cells)

In [ ]:
t_cells[["percent.mt"]] <- PercentageFeatureSet(t_cells, pattern = "^MT-")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
VlnPlot(t_cells, features = c('CD3E','CD3G','HBB','percent.mt'), pt.size=0, ncol = 2)

Let's drop outlier clusters: \
5, 13, & 16 for low of CD3 expression \
16 for high MT gene expression \
18 for high HBB expression 

In [ ]:
clean_t <- subset(t_cells, idents = c(5, 13, 16, 18), invert = TRUE)

In [ ]:
clean_t <- suppressWarnings(SCTransform(clean_t, conserve.memory = TRUE)) %>% RunPCA()

In [ ]:
clean_t <- RunUMAP(clean_t, dims=1:40) %>% FindNeighbors(dims=1:40) %>% FindClusters(resolution=0.5)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
DimPlot(clean_t, label = TRUE, repel = TRUE)
DimPlot(clean_t, group.by = 'predicted.celltype.l2')
DimPlot(clean_t, group.by = 'batch_id')
DimPlot(clean_t, group.by = 'age', shuffle = TRUE)

In [ ]:
saveRDS(clean_t, file = 'clean_aging_t_cells.rds')

In [ ]:
sessionInfo()